# Merge Collision and Weather Web-Scraped Data
This notebook is utilize to merge the web scraped data from Weather Underground and the NYC Collisions Dataset

In [1]:
# Import our libraries 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

In [ ]:
# Change file paths to match yours to utilize this notebook
df_weather = pd.read_csv('data/weather_2012_present.csv')
df_collisions_zones = pd.read_csv('data/collisions_and_zoningDistricts.csv')

In [54]:
df_weather['Weather_Datetime'] = pd.to_datetime(df_weather['date'] + ' ' + df_weather['time'])

C:\Users\Phoenix\AppData\Local\Temp\ipykernel_8720\2438155945.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather['Weather_Datetime'] = pd.to_datetime(df_weather['date'] + ' ' + df_weather['time'])


In [55]:
df_collisions_zones.head()

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,ZONEDIST
0,3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,Residential
1,4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,Park
2,6,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,...,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,Park
3,7,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,NaN,NaN,344 BAYCHESTER AVENUE,...,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,Mixed Manufacturing/Residential
4,8,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.897100,NaN,NaN,2047 PITKIN AVENUE,...,NaN,NaN,NaN,4487074,Sedan,NaN,NaN,NaN,NaN,Mixed Manufacturing/Residential


In [56]:
df_collisions_zones['Collision_Datetime'] = pd.to_datetime(df_collisions_zones['CRASH DATE'] + ' ' + df_collisions_zones['CRASH TIME'])

#Add description of each col and the reason for deleting the columns
useful_cols=['Collision_Datetime', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE','ON STREET NAME','CROSS STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2','COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'ZONEDIST']
df_c_1=df_collisions_zones[useful_cols]


drop_null_cols=['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE','CONTRIBUTING FACTOR VEHICLE 1','NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED','VEHICLE TYPE CODE 1']
df_c_2=df_c_1.dropna(subset=drop_null_cols).reset_index(drop=True)
print("Rows with non-null values in important columns:",df_c_2.shape[0])

Rows with non-null values in important columns: 1361902


In [57]:
acc_reason_df = pd.read_excel('data/contributing_factor.xlsx')

reason_dict = {}
for i in acc_reason_df.index:
  if acc_reason_df['broader_category'][i] in reason_dict:
    reason_dict[acc_reason_df['broader_category'][i]].append(acc_reason_df['contributing_factor'][i])
  else:
    reason_dict[acc_reason_df['broader_category'][i]] = [acc_reason_df['contributing_factor'][i]]

## Adding a new columns with a broader category of weather conditions
def get_categories(x):
  for k in reason_dict.keys():
    if x in reason_dict[k]:
      return k

df_c_2['broader_contributing_factors'] = df_c_2['CONTRIBUTING FACTOR VEHICLE 1'].apply(get_categories)
df_c_2=df_c_2.dropna(subset=['broader_contributing_factors']).reset_index(drop=True)    
df_c_2.head()

In [61]:
weather_useful_cols=['Weather_Datetime','date','time','temperature', 'dew_point', 'humidity', 
       'wind_speed', 'wind_gust', 'pressure', 'precip', 'condition']

df_w_1 = df_weather[weather_useful_cols]
df_w_1.head()

,Weather_Datetime,date,time,temperature,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition
0,2012-01-01 23:51:00,2012-01-01,11:51 PM,47 °F,37 °F,68 °%,12 °mph,0 °mph,30.00 °in,0.0 °in,Fair
1,2012-01-01 00:51:00,2012-01-01,12:51 AM,46 °F,37 °F,71 °%,12 °mph,0 °mph,30.01 °in,0.0 °in,Fair
2,2012-01-01 01:51:00,2012-01-01,1:51 AM,46 °F,37 °F,71 °%,13 °mph,0 °mph,30.02 °in,0.0 °in,Fair
3,2012-01-01 02:51:00,2012-01-01,2:51 AM,46 °F,37 °F,71 °%,10 °mph,0 °mph,30.03 °in,0.0 °in,Fair
4,2012-01-01 03:51:00,2012-01-01,3:51 AM,45 °F,36 °F,71 °%,3 °mph,0 °mph,30.02 °in,0.0 °in,Fair


In [62]:
def engineer_int_float_strings(numerical_string):
    numerical_string = str(numerical_string).strip()
    float_val = None
    try:
        float_val=float(re.findall("([0-9.-]*).*?",numerical_string)[0])
        #print(float_val)
    except Exception as e:
        print(e,numerical_string)
    return float_val

def standardize_weather_cols(row_x):
    row_x['temperature'] = engineer_int_float_strings(str(row_x['temperature']))
    row_x['dew_point'] = engineer_int_float_strings(str(row_x['dew_point']))
    row_x['humidity'] = engineer_int_float_strings(str(row_x['humidity']))
    row_x['wind_speed'] = engineer_int_float_strings(str(row_x['wind_speed']))
    row_x['wind_gust'] = engineer_int_float_strings(str(row_x['wind_gust']))
    row_x['pressure'] = engineer_int_float_strings(str(row_x['pressure']))
    row_x['precip'] = engineer_int_float_strings(str(row_x['precip']))
    return row_x

In [65]:
df_w_2 = df_w_1.apply(lambda x:standardize_weather_cols(x),axis=1)
df_w_2.head()

,Weather_Datetime,date,time,temperature,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition
0,2012-01-01 23:51:00,2012-01-01,11:51 PM,47.0,37.0,68.0,12.0,0.0,30.00,0.0,Fair
1,2012-01-01 00:51:00,2012-01-01,12:51 AM,46.0,37.0,71.0,12.0,0.0,30.01,0.0,Fair
2,2012-01-01 01:51:00,2012-01-01,1:51 AM,46.0,37.0,71.0,13.0,0.0,30.02,0.0,Fair
3,2012-01-01 02:51:00,2012-01-01,2:51 AM,46.0,37.0,71.0,10.0,0.0,30.03,0.0,Fair
4,2012-01-01 03:51:00,2012-01-01,3:51 AM,45.0,36.0,71.0,3.0,0.0,30.02,0.0,Fair


In [66]:
df_condition=pd.read_csv("data/conditions.csv")
df_condition.drop_duplicates(keep = 'first', inplace = True)
df_condition.reset_index(drop = True, inplace=True)

cond_dict = {}
for i in df_condition.index:
  if df_condition['broader_category'][i] in cond_dict:
    cond_dict[df_condition['broader_category'][i]].append(df_condition['condition'][i])
  else:
    cond_dict[df_condition['broader_category'][i]] = [df_condition['condition'][i]]

In [80]:
df_w_2.head()

,Weather_Datetime,date,time,temperature,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition,WeatherNearestHour
0,2012-01-01 23:51:00,2012-01-01,11:51 PM,47.0,37.0,68.0,12.0,0.0,30.00,0.0,Fair,2012-01-02 00:00:00
1,2012-01-01 00:51:00,2012-01-01,12:51 AM,46.0,37.0,71.0,12.0,0.0,30.01,0.0,Fair,2012-01-01 01:00:00
2,2012-01-01 01:51:00,2012-01-01,1:51 AM,46.0,37.0,71.0,13.0,0.0,30.02,0.0,Fair,2012-01-01 02:00:00
3,2012-01-01 02:51:00,2012-01-01,2:51 AM,46.0,37.0,71.0,10.0,0.0,30.03,0.0,Fair,2012-01-01 03:00:00
4,2012-01-01 03:51:00,2012-01-01,3:51 AM,45.0,36.0,71.0,3.0,0.0,30.02,0.0,Fair,2012-01-01 04:00:00


In [81]:
df_c_2.head()

,Collision_Datetime,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,ZONEDIST,broader_contributing_factors,CollisionNearestHour
0,2021-09-11 09:35:00,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.86650,NaN,NaN,0.0,...,0,0,Unspecified,NaN,4456314,Sedan,NaN,Residential,Unspecified,2021-09-11 10:00:00
1,2021-12-14 08:17:00,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.83148,NaN,NaN,2.0,...,2,0,Unspecified,Unspecified,4486660,Sedan,Sedan,Mixed Manufacturing/Residential,Unspecified,2021-12-14 08:00:00
2,2021-12-14 21:10:00,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.89710,NaN,NaN,0.0,...,0,0,Driver Inexperience,Unspecified,4487074,Sedan,NaN,Mixed Manufacturing/Residential,Driver Negligence,2021-12-14 21:00:00
3,2021-12-14 14:58:00,2021-12-14,14:58,MANHATTAN,10017.0,40.751440,-73.97397,3 AVENUE,EAST 43 STREET,0.0,...,0,0,Passing Too Closely,Unspecified,4486519,Sedan,Station Wagon/Sport Utility Vehicle,Commercial,Driver Negligence,2021-12-14 15:00:00
4,2021-12-14 16:50:00,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.75577,SPRINGFIELD BOULEVARD,EAST GATE PLAZA,0.0,...,0,0,Turning Improperly,Unspecified,4487127,Sedan,Station Wagon/Sport Utility Vehicle,Mixed Manufacturing/Residential,Driver Negligence,2021-12-14 17:00:00


In [79]:
# Round to the nearest hour while keeping the date intact
df_c_2['CollisionNearestHour'] = df_c_2['Collision_Datetime'] + pd.to_timedelta('30min')
df_c_2['CollisionNearestHour'] = df_c_2['CollisionNearestHour'].dt.floor('H')

df_w_2['WeatherNearestHour'] = df_w_2['Weather_Datetime'] + pd.to_timedelta('30min')
df_w_2['WeatherNearestHour'] = df_w_2['WeatherNearestHour'].dt.floor('H')


In [96]:
df_condition=pd.read_csv("data/conditions.csv")
df_condition.drop_duplicates(keep = 'first', inplace = True)
df_condition.reset_index(drop = True, inplace=True)
cond_dict = {}
for i in df_condition.index:
  if df_condition['broader_category'][i] in cond_dict:
    cond_dict[df_condition['broader_category'][i]].append(df_condition['condition'][i])
  else:
    cond_dict[df_condition['broader_category'][i]] = [df_condition['condition'][i]]

In [99]:
df_w_2.dropna(subset=['condition'],inplace=True)

In [100]:
## Adding a new columns with a broader category of weather conditions
def get_categories(x):
  if any(cat in x for cat in cond_dict['Snow']):
    return 'Snow'
  elif any(cat in x for cat in cond_dict['T-Storm']):
    return 'T-Storm'
  elif any(cat in x for cat in cond_dict['Rain']):
    return 'Rain'
  elif any(cat in x for cat in cond_dict['Fog']):
    return 'Fog'
  elif any(cat in x for cat in cond_dict['Windy']):
    return 'Windy'
  elif any(cat in x for cat in cond_dict['Cloudy']):
    return 'Cloudy'
  elif any(cat in x for cat in cond_dict['Fair']):
    return 'Fair'

df_w_2['broader_condition'] = df_w_2['condition'].apply(get_categories)

In [101]:
df_combined_2=df_c_2.merge(df_w_2,how='left',left_on="CollisionNearestHour",right_on="WeatherNearestHour",indicator=True)
df_combined_2.head()

,Collision_Datetime,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition,WeatherNearestHour,broader_condition,_merge
0,2021-09-11 09:35:00,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.86650,NaN,NaN,0.0,...,47.0,37.0,7.0,0.0,30.17,0.0,Fair,2021-09-11 10:00:00,Fair,both
1,2021-12-14 08:17:00,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.83148,NaN,NaN,2.0,...,28.0,46.0,13.0,0.0,30.40,0.0,Partly Cloudy,2021-12-14 08:00:00,Cloudy,both
2,2021-12-14 21:10:00,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.89710,NaN,NaN,0.0,...,19.0,33.0,7.0,0.0,30.60,0.0,Partly Cloudy,2021-12-14 21:00:00,Cloudy,both
3,2021-12-14 14:58:00,2021-12-14,14:58,MANHATTAN,10017.0,40.751440,-73.97397,3 AVENUE,EAST 43 STREET,0.0,...,18.0,25.0,9.0,0.0,30.49,0.0,Partly Cloudy,2021-12-14 15:00:00,Cloudy,both
4,2021-12-14 16:50:00,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.75577,SPRINGFIELD BOULEVARD,EAST GATE PLAZA,0.0,...,14.0,23.0,8.0,0.0,30.54,0.0,Partly Cloudy,2021-12-14 17:00:00,Cloudy,both


In [103]:
df_not_matched=df_combined_2[df_combined_2['_merge']!="both"]
df_not_matched.head()

,Collision_Datetime,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition,WeatherNearestHour,broader_condition,_merge
8939,2021-12-20 09:38:00,2021-12-20,9:38,QUEENS,11367.0,40.732390,-73.811160,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
9617,2021-12-20 09:45:00,2021-12-20,9:45,BROOKLYN,11201.0,40.696033,-73.984535,TILLARY STREET,FLATBUSH AVENUE EXTENSION,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
9642,2021-12-20 10:10:00,2021-12-20,10:10,MANHATTAN,10027.0,40.813828,-73.951935,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
9651,2021-12-20 09:45:00,2021-12-20,9:45,BROOKLYN,11201.0,40.696130,-73.987114,TILLARY STREET,JAY STREET,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
9694,2021-12-20 09:36:00,2021-12-20,9:36,QUEENS,11357.0,40.791985,-73.817220,149 STREET,10 AVENUE,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only


In [104]:
print("Number of rows not matched:",df_not_matched.shape[0])

Number of rows not matched: 2799


In [105]:
df_cw=df_combined_2[df_combined_2['_merge']=="both"].reset_index(drop=True)
df_cw.head()

,Collision_Datetime,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,dew_point,humidity,wind_speed,wind_gust,pressure,precip,condition,WeatherNearestHour,broader_condition,_merge
0,2021-09-11 09:35:00,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.86650,NaN,NaN,0.0,...,47.0,37.0,7.0,0.0,30.17,0.0,Fair,2021-09-11 10:00:00,Fair,both
1,2021-12-14 08:17:00,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.83148,NaN,NaN,2.0,...,28.0,46.0,13.0,0.0,30.40,0.0,Partly Cloudy,2021-12-14 08:00:00,Cloudy,both
2,2021-12-14 21:10:00,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.89710,NaN,NaN,0.0,...,19.0,33.0,7.0,0.0,30.60,0.0,Partly Cloudy,2021-12-14 21:00:00,Cloudy,both
3,2021-12-14 14:58:00,2021-12-14,14:58,MANHATTAN,10017.0,40.751440,-73.97397,3 AVENUE,EAST 43 STREET,0.0,...,18.0,25.0,9.0,0.0,30.49,0.0,Partly Cloudy,2021-12-14 15:00:00,Cloudy,both
4,2021-12-14 16:50:00,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.75577,SPRINGFIELD BOULEVARD,EAST GATE PLAZA,0.0,...,14.0,23.0,8.0,0.0,30.54,0.0,Partly Cloudy,2021-12-14 17:00:00,Cloudy,both


In [107]:
df_cw['broader_condition'].value_counts()   

broader_condition
Cloudy     1009202
Fair        263460
Rain        161288
Windy        84208
Snow         39584
Fog          30240
T-Storm      19429
Name: count, dtype: int64

In [ ]:
#Output CSV file
df_cw.to_csv("CollisionZonesWeather.csv")